In [20]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/data",one_hot=True)

Extracting /data\train-images-idx3-ubyte.gz
Extracting /data\train-labels-idx1-ubyte.gz
Extracting /data\t10k-images-idx3-ubyte.gz
Extracting /data\t10k-labels-idx1-ubyte.gz


In [21]:
element_size=28
time_steps=28
num_classes=10
batch_size=128
hidden_layer_size=128

LOG_DIR="logs/RNN_with_summaries"
_input=tf.placeholder(tf.float32, shape=[None,time_steps,element_size],name='inputs')
y=tf.placeholder(tf.float32, shape=[None,num_classes],name='labels')

batch_x, batch_y=mnist.train.next_batch(batch_size)
batch_x=batch_x.reshape((batch_size,time_steps,element_size))

In [22]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean)
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev)
        tf.summary.scalar('max',tf.reduce_max(var))
        tf.summary.scalar('min',tf.reduce_min(var))
        tf.summary.histogram('histogram',var)
        

In [23]:
with tf.name_scope('rnn_weights'):
    with tf.name_scope("W_x"):
        Wx=tf.Variable(tf.zeros([element_size, hidden_layer_size]))
        variable_summaries(Wx)
    with tf.name_scope("W_h"):
        Wh=tf.Variable(tf.zeros([hidden_layer_size, hidden_layer_size]))
        variable_summaries(Wh)
    with tf.name_scope("Bias"):
        b_rnn=tf.Variable(tf.zeros([hidden_layer_size]))
        variable_summaries(b_rnn)
    

In [24]:
def rnn_step(previous_hidden_state,x):
    current_hidden_state=tf.tanh(tf.matmul(previous_hidden_state,Wh)+tf.matmul(x,Wx)+b_rnn)
    return current_hidden_state

In [25]:
processed_input=tf.transpose(_input,perm=[1,0,2])
initial_hidden=tf.zeros([batch_size, hidden_layer_size])
all_hidden_states=tf.scan(rnn_step,processed_input, initializer=initial_hidden,name='states')

In [26]:
with tf.name_scope('linear_layer_weights') as scope:
    with tf.name_scope("W_linear"):
        Wl=tf.Variable(tf.truncated_normal([hidden_layer_size,num_classes], mean=0, stddev=.01))
        variable_summaries(Wl)
    with tf.name_scope("Bias_linear"):
        bl=tf.Variable(tf.truncated_normal([num_classes],mean=0,stddev=.01))
        variable_summaries(bl)
def get_linear_layer(hidden_state):
    return tf.matmul(hidden_state,Wl)+bl
with tf.name_scope('linear_layer_weights') as scope:
    all_outputs=tf.map_fn(get_linear_layer, all_hidden_states)
    output=all_outputs[-1]
    tf.summary.histogram('outputs',output)

In [27]:
with tf.name_scope('cross_entropy'):
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=y))
    tf.summary.scalar('cross_entropy',cross_entropy)
with tf.name_scope('train'):
    train_step=tf.train.RMSPropOptimizer(0.001,0.9).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(output,1))
    accuracy=(tf.reduce_mean(tf.cast(correct_prediction,tf.float32)))*100
    tf.summary.scalar('accureacy',accuracy)
merged=tf.summary.merge_all()

In [28]:
test_data=mnist.test.images[:batch_size].reshape((-1,time_steps,element_size))
test_label=mnist.test.labels[:batch_size]

In [41]:
with tf.Session() as sess:
    train_writer=tf.summary.FileWriter(LOG_DIR+'/train',graph=tf.get_default_graph())
    test_writer=tf.summary.FileWriter(LOG_DIR+'/test',graph=tf.get_default_graph())
    sess.run(tf.global_variables_initializer())
    for i in range(10000):
        batch_x, batch_y=mnist.train.next_batch(batch_size)
        batch_x=batch_x.reshape((batch_size,time_steps,element_size))
        sess.run([train_step],feed_dict={_input:batch_x,y:batch_y})
        #train_writer.add_summary(summary,i)
        if i%1000==0:
            acc,loss, = sess.run([accuracy,cross_entropy],feed_dict={_input:batch_x,y:batch_y})
            print("Iter "+str(i)+", Minibatch Loss= "+"{:.6f}".format(loss)+", Training Accuracy= "+"{:.5f}".format(acc))
        #if i%10:
            #summary, acc=sess.run([merged,accuracy],feed_dict={_input:test_data, y:test_label})
            #test_writer.add_summary(summary,i)
    test_acc=sess.run(accuracy,feed_dict={_input:test_data, y:test_label})
    print("Test Accuracy:",test_acc)

Iter 0, Minibatch Loss= 2.303116, Training Accuracy= 5.46875
Iter 1000, Minibatch Loss= 1.394797, Training Accuracy= 43.75000
Iter 2000, Minibatch Loss= 0.592246, Training Accuracy= 82.03125
Iter 3000, Minibatch Loss= 0.393879, Training Accuracy= 92.18750
Iter 4000, Minibatch Loss= 0.148196, Training Accuracy= 93.75000
Iter 5000, Minibatch Loss= 0.198646, Training Accuracy= 94.53125
Iter 6000, Minibatch Loss= 0.205712, Training Accuracy= 94.53125
Iter 7000, Minibatch Loss= 0.082582, Training Accuracy= 98.43750
Iter 8000, Minibatch Loss= 0.056843, Training Accuracy= 98.43750
Iter 9000, Minibatch Loss= 0.075027, Training Accuracy= 97.65625
Test Accuracy: 97.65625
